In [31]:
import pandas as pd
import glob
import re
import numpy as np

In [32]:
path = '/Users/scai/CMRI/tmm/tel_vs_alt/data/freec/somatic/*bam_ratio.txt'

In [33]:
samples = pd.read_csv("../data/samples/SAMPLES_Excel_JFCF.tsv", sep='\t')
samples_ALT = samples[samples['TMM'] == 'ALT']['Immortal']
samples_TEL = samples[samples['TMM'] == 'TEL']['Immortal']

In [34]:
files = {}

In [35]:
for fp in glob.glob(path):
    name = re.search('.*(JFCF.*)\.bam.*', fp).group(1).replace('-','_')
    if name == "JFCF_6_T_1_P":
        name = "JFCF_6_T_1_P_TEL"
    if name == "JFCF_6_T_1_PALT":
        name = "JFCF_6_T_1_P_ALT"
    if name == "JFCF6_T_1J_1_3C":
        name = "JFCF_6_T_1J_1_3C"
    if name == "JFCF6_T_1J_6B":
        name = "JFCF_6_T_1J_6B"
    if name == "JFCF_61M":
        name = "JFCF_6_T_1_M"
    if name == "JFCF_61Q":
        name = "JFCF_6_T_1_Q"
    if name == "JFCF_6_p15_marked":
        name = "JFCF_6_p15"
    print(name)
    files[name] = fp

JFCF_6_T_1_G
JFCF_6_T_1J_11E
JFCF_6_T_1_C
JFCF_6_T_1_R
JFCF_6_T_1_D
JFCF_6_T_5K
JFCF_6_T_2H
JFCF_6_T_1_L
JFCF_6_T_1_H
JFCF_6_T_1_P_ALT
JFCF_6_T_1_M
JFCF_6_T_1_P_TEL
JFCF_6_T_1_Q
JFCF_6_T_1_F


In [36]:
df_all = []

In [37]:
for key, value in files.items():
    df = pd.read_csv(value, sep='\t')
    df['sample'] = key
    df_all.append(df)
df_all = pd.concat(df_all)

In [38]:
df_all['logR'] = df_all.apply(
    lambda row: 0 if row['MedianRatio'] < 0 else np.log2(row['MedianRatio'])
    if row['Chromosome'] not in
#     ('X', 'Y') else np.log2(row['MedianRatio']) + 1,
    ('X', 'Y') else np.log2(row['MedianRatio']),
    axis=1)

In [39]:
df_all['Chromosome'] = df_all['Chromosome'].str.replace('X', '23')
df_all['Chromosome'] = df_all['Chromosome'].str.replace('Y', '24')

df_all = df_all[df_all['Chromosome'].isin([str(x) for x in range(1, 25)])]

df_all = df_all.astype({'Chromosome': int})

df_all['Chromosome'].unique()

df_all = df_all[['sample', 'Chromosome', 'Start',
                 'logR']].sort_values(by=['sample', 'Chromosome', 'Start'])

In [40]:
df_tel = df_all[df_all['sample'].isin(samples_TEL) | (df_all['sample']=='JFCF_6_p15')]
df_tel.to_csv('../data/freec_tel_somatic.csv', index=False)

df_alt = df_all[(df_all['sample'].isin(samples_ALT)) | (df_all['sample']=='JFCF_6_p15')]
df_alt.to_csv('../data/freec_alt_somatic.csv', index=False)


In [ ]:

df_mortal = df_all[df_all['sample']=='JFCF_6_p15']
df_mortal.to_csv('../data/freec_mortal.csv', index=False)